In [39]:
from trailmet.algorithms.distill.attention_transfer import AttentionTransfer
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

# Dataset

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Pad(4, padding_mode='reflect'),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomCrop(32),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transforms1 = {
    'train': transform_train, 
    'val': transform_test, 
    'test': transform_test}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

In [42]:
from trailmet.datasets.classification import DatasetFactory

cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = "./data",
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.1,
                                        transform = transforms1,
                                        target_transform = target_transforms,
                                        random_seed=42
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [43]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=128, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=2
    )

val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=128, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=2
    )

test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test_dataset'], batch_size=128, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=2
    )

In [44]:
dataloaders = {'train':train_loader, 'val':val_loader, 'test':test_loader}

# Model

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        out1 = self.block1(out)
        out2 = self.block2(out1)
        out3 = self.block3(out2)
        out = self.relu(self.bn1(out3))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)

        return self.fc(out)


def make_wideresnet(depth, widen_factor, num_classes, dropRate=0.0):
    """Creates and returns the requested model

    Parameters
    ----------
    depth : int
    widen_factor : int
    num_classes : int
    dropRate : float, optional

    Returns
    -------
    WideResNet model

    """
    model = WideResNet(depth, num_classes, widen_factor, dropRate)
    return model

In [46]:
teacher_model = make_wideresnet(16, 2, 10)
student_model = make_wideresnet(16, 1, 10)

teacher_model.to(device)
student_model.to(device)

WideResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): NetworkBlock(
    (layer): Sequential(
      (0): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(

In [47]:
teacher_model.load_state_dict(torch.load('./wrn-16-2-(94.02).pt'))

<All keys matched successfully>

In [48]:
kwargs = {
    'DISTILL_ARGS' : {
        'BETA':1000,
        'EPOCHS':200,
        'LR':0.1,
        'WEIGHT_DECAY':0.0005,
        'TEACHER_LAYER_NAMES':['block1', 'block2', 'block3'],
        'STUDENT_LAYER_NAMES':['block1', 'block2', 'block3'],
    },
    'DEVICE' : device,
    'log_dir' : 'at_wrn16x1'
}

In [49]:
box = AttentionTransfer(teacher_model, student_model, dataloaders, **kwargs)

In [50]:
box.compress_model()

=====TRAINING STUDENT NETWORK=====
Epoch: 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 41.25it/s, acc=0.589, loss=2.73]


**Saving checkpoint**
Epoch: 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 38.35it/s, acc=0.706, loss=2.36]


**Saving checkpoint**
Epoch: 3


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 39.56it/s, acc=0.924, loss=0.909]


Epoch: 199


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 37.72it/s, acc=0.924, loss=0.908]


Epoch: 200


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 38.24it/s, acc=0.925, loss=0.904]


In [48]:
student_model.load_state_dict(torch.load('./checkpoints/at_wrn16x1.pth')['state_dict'])

<All keys matched successfully>

In [51]:
preds = []
valid_labels = []
student_model.eval()
# Run the best model on test set
for step, (images, labels) in tqdm(enumerate(test_loader), total = len(test_loader)):

    images = images.to(device, dtype=torch.float)
    labels = labels.to(device)
    batch_size = labels.size(0)

    with torch.no_grad():
        y_preds = student_model(images)
        
    preds.append(y_preds.softmax(1).to('cpu').numpy())
    valid_labels.append(labels.to('cpu').numpy())

predictions = np.concatenate(preds)
valid_labels = np.concatenate(valid_labels)

# Get the score
score = accuracy_score(valid_labels, predictions.argmax(1))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 52.71it/s]


In [52]:
score

0.9207